# Imports 

In [1]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` (North American Industry Classification Systems)

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [2]:
# uncomment if u need to find your path: os.getcwd()
SD = sd_df = pd.read_csv("../public_data/sd_df.csv")
NA = naics_df = pd.read_csv("../public_data/naics_df.csv")

In [3]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes
sd_df.head()
naics_df.head()

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

account_key                                      dba_name council_district  \
0  1974000448                             ERNST & YOUNG LLP             cd_1   
1  1974011093  HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP             cd_3   
2  1978039819                                    RSM US LLP             cd_1   
3  1978042092               THORSNES BARTOLOTTA MCGUIRE LLP             cd_3   
4  1979046817                       KORENIC & WOJDOWSKI LLP             cd_7   

  naics_code                              naics_description  naics_nchar  
0     541211        OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS            6  
1       5411                                 LEGAL SERVICES            4  
2     541211        OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS            6  
3       5411                                 LEGAL SERVICES            4  
4       5412  ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES            4

naics                 naics_description
0  111140                     Wheat Farming
1  111160                      Rice Farming
2  111150                      Corn Farming
3  111110                   Soybean Farming
4  111120  Oilseed (except Soybean) Farming

## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [7]:
sd_merged = SD.merge(
    NA,
    left_on="naics_code",
    right_on="naics",
    how="inner"
)


matched_naics = set(sd_merged['naics_code'])


sd_lost = SD[~SD['naics_code'].isin(matched_naics)]

print("Examples of San Diego businesses lost in the merge:")
print(sd_lost.head())

# Step 3: Use value_counts() on naics_nchar in the original data
print("\nValue counts of naics_nchar in original San Diego data:")
print(SD['naics_nchar'].value_counts())


Examples of San Diego businesses lost in the merge:
  account_key                                      dba_name council_district  \
1  1974011093  HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP             cd_3   
3  1978042092               THORSNES BARTOLOTTA MCGUIRE LLP             cd_3   
4  1979046817                       KORENIC & WOJDOWSKI LLP             cd_7   
5  1979053082    GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP             cd_3   
6  1981000840                          BENINK & SLAVENS LLP             cd_7   

  naics_code                              naics_description  naics_nchar  
1       5411                                 LEGAL SERVICES            4  
3       5411                                 LEGAL SERVICES            4  
4       5412  ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES            4  
5       5411                                 LEGAL SERVICES            4  
6       5411                                 LEGAL SERVICES            4  

Value counts of 

## "Left join"- retain all sd businesses even if naics code isn't in `naics_df`

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix
- Use `naics_merge_status` in the merged result to look at a sample of San Diego businesses that weren't matched with `naics_df`

In [9]:
joined_left = SD.merge(
    NA,
    left_on="naics_code",
    right_on = "naics",
    indicator = "naics_merge_status",
    how = "left",
    suffixes = ("_sd", "_census")
)

unmatched_sd = joined_left[joined_left["naics_merge_status"] == "left_only"]
unmatched_sd

account_key                                      dba_name  \
1    1974011093  HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP   
3    1978042092               THORSNES BARTOLOTTA MCGUIRE LLP   
4    1979046817                       KORENIC & WOJDOWSKI LLP   
5    1979053082    GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP   
6    1981000840                          BENINK & SLAVENS LLP   
..          ...                                           ...   
362  2006015404                  9625 TOWN CENTRE PARTNERS LP   
363  2007015303                                WFP-GATEWAY LP   
364  2007015336                                WFP-MIRAMAR LP   
365  2010010820         ROSSLYN LOFTS HOUSING PARTNERS LP THE   
369  2014014569                           JLMR INVESTMENTS LP   

    council_district naics_code  \
1               cd_3       5411   
3               cd_3       5411   
4               cd_7       5412   
5               cd_3       5411   
6               cd_7       5411   
..               ...        ...   
362             cd_3        541   
363             cd_3        541   
364             cd_3        541   
365             cd_6        541   
369             cd_2        531   

                              naics_description_sd  naics_nchar naics  \
1                                   LEGAL SERVICES            4   NaN   
3                                   LEGAL SERVICES            4   NaN   
4    ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES            4   NaN   
5                                   LEGAL SERVICES            4   NaN   
6                                   LEGAL SERVICES            4   NaN   
..                                             ...          ...   ...   
362  PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES            3   NaN   
363  PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES            3   NaN   
364  PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES            3   NaN   
365  PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES            3   NaN   
369                                    REAL ESTATE            3   NaN   

    naics_description_census naics_merge_status  
1                        NaN          left_only  
3                        NaN          left_only  
4                        NaN          left_only  
5                        NaN          left_only  
6                        NaN          left_only  
..                       ...                ...  
362                      NaN          left_only  
363                      NaN          left_only  
364                      NaN          left_only  
365                      NaN          left_only  
369                      NaN          left_only  

[318 rows x 9 columns]

## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator, `is_lost`, that equals `True` if the merge indicator is equal to "left_only" (and is otherwise false)
- Find the proportion of businesses by council district that were lost in the left join (SD businesses that failed to match to `naics_df`). To do this, group by `council_district` and use the shortcut of taking the mean of the `is_lost` indicator to find the proportions lost by `council_district`. 

In [16]:
joined_left["is_lost"] = joined_left["naics_merge_status"] == "left_only"

lost_proportions = joined_left.groupby("council_district").agg({"is_lost": "mean"})

lost_proportions

is_lost
council_district          
cd_1              0.880597
cd_2              0.804348
cd_3              0.812500
cd_4              1.000000
cd_5              0.916667
cd_6              0.859649
cd_7              0.918919
cd_8              0.933333
cd_9              1.000000

## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were often less than six digits long, while the Census ones were always 6 digits.

You wonder if this is an issue where 0's in some of the SD's data naics codes got cut off (eg 540000 became 54), and if so whether adding these lagging zeros would improve the merge rate in a left join.

- Pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits, using one of two approaches: 
  1. `str.pad` in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html)
  2. for more of a challenge, write your own function! It should check the # of digits in the naics code string and pad it with 0's at the end up to 6 digits. To execute your function, use row-wise apply: `df.apply(lambda row: funcname(row.column), axis=1)`.
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [ ]:
padded_sd = sd_df['naics_code'].astype(str).str.pad(6, side='right', fillchar='0')

sd_df["padded_naics_code"] = padded_sd
left_challenge_df = sd_df.merge(naics_df, left_on = "padded_naics_code", right_on = "naics", how = "left", indicator = True, suffixes = ("_sd", "_census"))
left_challenge_df.rename(columns = {"_merge":"naics_merge_status"}, inplace = True)
#left_challenge_df.head()

left_challenge_df["is_lost"] = left_challenge_df["naics_merge_status"] == "left_only"
#left_challenge_df.head()


## COMPARING (with last match?)
left_df.groupby("council_district").agg({"is_lost": "mean"})
left_challenge_df.groupby("council_district").agg({"is_lost": "mean"}) 


## compare the naics_descrption from SD vs census in the new dataset for a sample of these new matches

left_challenge_df[["naics_description_sd", "naics_description_census"]]
left_df[["naics_description_sd", "naics_description_census"]]